<a href="https://colab.research.google.com/github/ramkumardeepak774/Text_Classification_Flair-vs-Fasttext/blob/main/Classification_of_sysco_and_idfc_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
#upload data from local
from google.colab import files
idfc_train = files.upload()
idfc_test = files.upload()

Saving idfc-email-train.csv to idfc-email-train.csv


Saving idfc-email-test.csv to idfc-email-test.csv


Data preprocessing and visulization


In [3]:
cisco_data = files.upload()

Saving sysco-full.csv to sysco-full.csv


In [4]:
import io
idfc_train = pd.read_csv(io.BytesIO(idfc_train["idfc-email-train.csv"]))
idfc_test = pd.read_csv(io.BytesIO(idfc_test["idfc-email-test.csv"]))

In [5]:
print(idfc_train.shape)
print(idfc_test.shape)
idfc_train.head()

(1440, 2)
(360, 2)


,SubCategory,Subject Description
0,Application Rejected,NA Help
1,Application Rejected,Credit card status R/SirI applied for credit c...
2,Application Rejected,Regarding showing secured credit card I...
3,Application Rejected,"Credit card application reg,. Dear sir,I tried..."
4,Application Rejected,NA Where is my credit card


In [6]:
cisco_data = pd.read_csv(io.BytesIO(cisco_data["sysco-full.csv"]))

In [7]:
print(cisco_data.shape)
cisco_data.head()

(23897, 2)


,Description,Request Type
0,Good afternoon. There was a delay in getting t...,General Inquiry
1,"Hello, We received the below deduction from yo...",Purchase Order
2,"Hi, Per attached Invoice, correct Pickup Allow...",Purchase Order
3,"Good afternoon, Sysco Florida South Co-op shor...",Purchase Order
4,"Good afternoon, Sysco Florida South Co-op shor...",Purchase Order


In [8]:
#change the order of column in idfc data
idfc_train = idfc_train .iloc[:,[1,0]]
idfc_test = idfc_test.iloc[:, [1,0]]

In [9]:
idfc_train.columns  = ['text', 'label']
idfc_test.columns  = ['text', 'label']
cisco_data.columns = ['text', 'label']

In [10]:
idfc_train.label.value_counts()

Application Rejected       240
Card Application Status    240
Debit Card Offers          240
Fund Transfer              240
Matured Loan Closure       240
Other Bank Charges         240
Name: label, dtype: int64

In [11]:
idfc_test.label.value_counts()

Application Rejected       60
Card Application Status    60
Debit Card Offers          60
Fund Transfer              60
Matured Loan Closure       60
Other Bank Charges         60
Name: label, dtype: int64

In [12]:
cisco_data.label.value_counts()

Purchase Order      12124
General Inquiry      5538
Supplier Claim       2835
Repayment            2388
Supplier Fee          319
Corporate Billed        5
Name: label, dtype: int64

In [13]:
# In Cisco have unequal number of frequency 
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 69.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:0

In [15]:
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from flair.data import Sentence
from spacy.lang.en import English


In [16]:
lemmatizer = WordNetLemmatizer()


In [17]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [18]:
from spacy.lang.en.stop_words import STOP_WORDS


In [19]:

# function for text preprocessing
def preprocess_text(text):
    # convert text to lowercase
    if isinstance(text, str):
      text = text.lower()
    else :
      text = str(text).lower()
    
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # remove numbers
    text = re.sub(r'\d+', '', text)
    

    my_doc = nlp(text)
    filtered_sentence =[] 

    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    # join tokens back to text
    text = ' '.join(filtered_sentence)
    
    return text

In [20]:

# apply text preprocessing to idfc train data
idfc_train['text'] = idfc_train['text'].apply(preprocess_text)

In [21]:
# apply text preprocessing to idfc test data
idfc_test['text'] = idfc_test['text'].apply(preprocess_text)

In [22]:
# apply text preprocessing to cisco data
cisco_data['text'] = cisco_data['text'].apply(preprocess_text)



In [23]:
cisco_data.head()

,text,label
0,good afternoon delay getting pricing corrected...,General Inquiry
1,hello received deduction determined invalid ...,Purchase Order
2,hi attached invoice correct pickup allowance f...,Purchase Order
3,good afternoon sysco florida south coop short ...,Purchase Order
4,good afternoon sysco florida south coop short ...,Purchase Order


##classification of idfc data using fasttext

In [24]:
!pip install fasttext


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393352 sha256=bbfad6b47c59258b861e68982da0adedf5a944a6d6482860cfb25101c7b74101
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [25]:
import fasttext

In [26]:
idfc_train_fasttext = idfc_train.sample(frac = 1)
idfc_test_fasttext = idfc_test.sample(frac = 1)
cisco_fasttext = cisco_data.sample(frac = 1)

In [27]:
#Data prepare for fasttext format for train
idfc_train_fasttext["label_format"]=0
for i in range(len(idfc_train_fasttext)):
    idfc_train_fasttext.label_format[i]="__label__"+str(idfc_train_fasttext.label[i])+" "+str(idfc_train_fasttext.text[i])
     


<ipython-input-27-877f111890c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idfc_train_fasttext.label_format[i]="__label__"+str(idfc_train_fasttext.label[i])+" "+str(idfc_train_fasttext.text[i])


In [28]:
idfc_train_fasttext.head()

,text,label,label_format
120,facing problem applying new credit card hi dr ...,Application Rejected,__label__Application Rejected facing problem a...
672,case voucher receive dear siri rajni like...,Debit Card Offers,__label__Debit Card Offers case voucher rec...
1205,ecs charges issue rekha r dear teami wheeler l...,Other Bank Charges,__label__Other Bank Charges ecs charges issue ...
1152,flipkart pay later closer request hello idfc t...,Matured Loan Closure,__label__Matured Loan Closure flipkart pay lat...
857,idfcfirstbankstatementpdf k fayaz,Fund Transfer,__label__Fund Transfer idfcfirstbankstatementp...


In [29]:
#Data prepare for fasttext format for test
idfc_test_fasttext["label_format"]=0
for i in range(len(idfc_test_fasttext)):
    idfc_test_fasttext.label_format[i]="__label__"+str(idfc_test_fasttext.label[i])+" "+str(idfc_test_fasttext.text[i])
     

<ipython-input-29-7bec91791cff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idfc_test_fasttext.label_format[i]="__label__"+str(idfc_test_fasttext.label[i])+" "+str(idfc_test_fasttext.text[i])


In [30]:
idfc_train_fasttext.label_format.to_csv('fasttext_train.txt',index=None,header=None)
idfc_test_fasttext.label_format.to_csv('fasttext_test.txt',index=None,header=None)


In [31]:
#train classifier model
model = fasttext.train_supervised('fasttext_train.txt',epoch=50,lr=0.05,label_prefix='__label__',dim=300)
     

In [32]:
# test classifier on testing data and get evaluation metrics
#test fasttext model
result = model.test('fasttext_test.txt')
print(result)

(360, 0.9944444444444445, 0.9944444444444445)


In [33]:
f1_score = (2*result[1]*result[2])/(result[1] +result[2])
print(f1_score)

0.9944444444444445


## fasttext classification on cisco data set



In [34]:
cisco_fasttext.head()

,text,label
707,hello remit repayment invoice attached por...,Purchase Order
22023,mims laurie lauriemimssyscocom sent monday jul...,Supplier Claim
4776,hi team assist request regardsrekha bishtsysco...,Purchase Order
20712,attached report send cost sell price items pai...,Purchase Order
5116,attached invoice po c direct questions payment...,Purchase Order


In [35]:
#Data prepare for fasttext format for train
cisco_fasttext["label_format"]=0
for i in range(len(cisco_fasttext)):
    cisco_fasttext.label_format[i]="__label__"+str(cisco_fasttext.label[i])+" "+str(cisco_fasttext.text[i])
     


<ipython-input-35-87799412a52b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cisco_fasttext.label_format[i]="__label__"+str(cisco_fasttext.label[i])+" "+str(cisco_fasttext.text[i])


In [36]:
cisco_fasttext.head()

,text,label,label_format
707,hello remit repayment invoice attached por...,Purchase Order,__label__Purchase Order hello remit repayment ...
22023,mims laurie lauriemimssyscocom sent monday jul...,Supplier Claim,__label__Supplier Claim mims laurie lauriemims...
4776,hi team assist request regardsrekha bishtsysco...,Purchase Order,__label__Purchase Order hi team assist request...
20712,attached report send cost sell price items pai...,Purchase Order,__label__Purchase Order attached report send c...
5116,attached invoice po c direct questions payment...,Purchase Order,__label__Purchase Order attached invoice po c ...


In [37]:
from sklearn.model_selection import train_test_split


In [38]:
train_data, test_data = train_test_split(cisco_fasttext, test_size=0.15, random_state=42)


In [39]:
train_data.label.value_counts()

Purchase Order      10303
General Inquiry      4702
Supplier Claim       2434
Repayment            2017
Supplier Fee          275
Corporate Billed        3
Name: label, dtype: int64

In [40]:
test_data.label.value_counts()

Purchase Order      1821
General Inquiry      836
Supplier Claim       401
Repayment            371
Supplier Fee          44
Corporate Billed       2
Name: label, dtype: int64

In [41]:
train_data.label_format.to_csv('fasttext_cisco_train.txt',index=None,header=None)
test_data.label_format.to_csv('fasttext_cisco_test.txt',index=None,header=None)


In [42]:
#train classifier model
model = fasttext.train_supervised('fasttext_cisco_train.txt',epoch=50,lr=0.05,label_prefix='__label__',dim=300)
     

In [43]:
#test fasttext model
result = model.test('fasttext_cisco_test.txt')
print(result)

(3585, 0.9935843793584379, 0.9935843793584379)


In [44]:
f1_score = (2*result[1]*result[2])/(result[1] +result[2])
print(f1_score)

0.9935843793584379


## flair classification on idfc data

In [65]:
# Create A Folder for the csv to make corpus for idfc data
!mkdir -p idfc_data_lstm
!mkdir -p idfc_data_sent

In [66]:
idfc_train.to_csv("idfc_data_lstm/train.csv")
idfc_test.to_csv("idfc_data_lstm/test.csv")

idfc_train.to_csv("idfc_data_sent/train.csv")
idfc_test.to_csv("idfc_data_sent/test.csv")

In [67]:
from flair.datasets import ClassificationCorpus,CSVClassificationCorpus
from flair.data import Corpus

In [68]:
# Create Column Mapping to show which column is for label and text
column_name_map = {2:"label",1:"text"}

In [62]:
# Location for CSV
data_folder = '/content/idfc_data_lstm'

In [72]:
# Create Corpus For CSV for model 1
corpus_csv1: Corpus = CSVClassificationCorpus('/content/idfc_data_sent',
                                             column_name_map=column_name_map,
                                             skip_header=True,
                                             label_type='label',
                                             delimiter=',')

2023-05-11 03:42:05,988 Reading data from /content/idfc_data_sent
2023-05-11 03:42:05,991 Train: /content/idfc_data_sent/train.csv
2023-05-11 03:42:05,993 Dev: None
2023-05-11 03:42:05,995 Test: /content/idfc_data_sent/test.csv


In [73]:
# Create Corpus For CSV for model 1
corpus_csv2: Corpus = CSVClassificationCorpus(data_folder,
                                             column_name_map=column_name_map,
                                             skip_header=True,
                                             label_type='label',
                                             delimiter=',')

2023-05-11 03:42:08,739 Reading data from /content/idfc_data_lstm
2023-05-11 03:42:08,742 Train: /content/idfc_data_lstm/train.csv
2023-05-11 03:42:08,749 Dev: None
2023-05-11 03:42:08,751 Test: /content/idfc_data_lstm/test.csv


In [74]:
# Creating the Label Diction For CSV
label_type='label'
label_dict_csv1 = corpus_csv1.make_label_dictionary(label_type= label_type)
label_dict_csv2 = corpus_csv2.make_label_dictionary(label_type= label_type)


2023-05-11 03:42:28,301 Computing label dictionary. Progress:


1296it [00:00, 1309.19it/s]

2023-05-11 03:42:29,299 Dictionary created for label 'label' with 7 values: Matured Loan Closure (seen 222 times), Debit Card Offers (seen 219 times), Other Bank Charges (seen 218 times), Card Application Status (seen 213 times), Fund Transfer (seen 213 times), Application Rejected (seen 211 times)
2023-05-11 03:42:29,303 Computing label dictionary. Progress:



1296it [00:00, 1368.96it/s]

2023-05-11 03:42:30,259 Dictionary created for label 'label' with 7 values: Debit Card Offers (seen 224 times), Application Rejected (seen 221 times), Fund Transfer (seen 220 times), Matured Loan Closure (seen 212 times), Card Application Status (seen 211 times), Other Bank Charges (seen 208 times)


In [45]:
# we want to train two model first on sentence transformer bert-base-nli-mean-tokens and second on transformerdocumentembedding distilbert-base-uncased
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=8999cb0b561ba8a865045c2fb7ca19c091ecdb1fe0aed533bab4fd8b260ce218
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [53]:
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.embeddings import TransformerDocumentEmbeddings

In [54]:
# 4. initialize transformer document embeddings (many models are available)

document_embeddings1 = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')
document_embeddings2 = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

In [75]:
# 5. create the text classifier
classifier1 = TextClassifier(document_embeddings1, label_dictionary=label_dict_csv1, multi_label=True, label_type="label")
classifier2 = TextClassifier(document_embeddings2, label_dictionary=label_dict_csv2, multi_label=True, label_type="label")

In [76]:
trainer1 = ModelTrainer(classifier1, corpus_csv1)
trainer2 = ModelTrainer(classifier2, corpus_csv2)

In [ ]:
trainer1.train('/content/idfc_data_sent',
              learning_rate=0.05,
              mini_batch_size=50,
              max_epochs=5)
trainer2.train('/content/idfc_data_lstm',
              learning_rate=0.05,
              mini_batch_size=50,
              max_epochs=5)

## Classification using flair on cisco data set

In [77]:
# Create A Folder for the csv to make corpus for idfc data
!mkdir -p cisco_data_lstm
!mkdir -p cisco_data_sent

In [78]:
cisco_train, cisco_test = train_test_split(cisco_data, test_size=0.15, random_state=42)


In [81]:
cisco_train.to_csv("cisco_data_lstm/train.csv")
cisco_test.to_csv("cisco_data_lstm/test.csv")

cisco_train.to_csv("cisco_data_sent/train.csv")
cisco_test.to_csv("cisco_data_sent/test.csv")

In [82]:
# Create Corpus For CSV for model 1
corpus_csv_cisco1: Corpus = CSVClassificationCorpus('/content/cisco_data_sent',
                                             column_name_map=column_name_map,
                                             skip_header=True,
                                             label_type='label',
                                             delimiter=',')

2023-05-11 03:54:59,369 Reading data from /content/cisco_data_sent
2023-05-11 03:54:59,374 Train: /content/cisco_data_sent/train.csv
2023-05-11 03:54:59,375 Dev: None
2023-05-11 03:54:59,378 Test: /content/cisco_data_sent/test.csv


In [83]:
# Create Corpus For CSV for model 1
corpus_csv_cisco2: Corpus = CSVClassificationCorpus('/content/cisco_data_lstm',
                                             column_name_map=column_name_map,
                                             skip_header=True,
                                             label_type='label',
                                             delimiter=',')

2023-05-11 03:55:24,280 Reading data from /content/cisco_data_lstm
2023-05-11 03:55:24,282 Train: /content/cisco_data_lstm/train.csv
2023-05-11 03:55:24,283 Dev: None
2023-05-11 03:55:24,286 Test: /content/cisco_data_lstm/test.csv


In [84]:
label_dict_csv_cisco1 = corpus_csv_cisco1.make_label_dictionary(label_type= label_type)
label_dict_csv_cisco2 = corpus_csv_cisco2.make_label_dictionary(label_type= label_type)

2023-05-11 03:58:01,134 Computing label dictionary. Progress:


17758it [00:45, 388.39it/s]

2023-05-11 03:58:46,874 Dictionary created for label 'label' with 7 values: Purchase Order (seen 9267 times), General Inquiry (seen 4275 times), Supplier Claim (seen 2146 times), Repayment (seen 1819 times), Supplier Fee (seen 246 times), Corporate Billed (seen 5 times)
2023-05-11 03:58:46,877 Computing label dictionary. Progress:



17758it [00:40, 433.30it/s]

2023-05-11 03:59:27,874 Dictionary created for label 'label' with 7 values: Purchase Order (seen 9264 times), General Inquiry (seen 4302 times), Supplier Claim (seen 2144 times), Repayment (seen 1798 times), Supplier Fee (seen 245 times), Corporate Billed (seen 5 times)


In [ ]:
# 5. create the text classifier
classifier1 = TextClassifier(document_embeddings1, label_dictionary=label_dict_csv_cisco1, multi_label=True, label_type="label")
classifier2 = TextClassifier(document_embeddings2, label_dictionary=label_dict_csv_cisco2, multi_label=True, label_type="label")

In [ ]:
trainer1 = ModelTrainer(classifier1, corpus_csv_cisco1)
trainer2 = ModelTrainer(classifier2, corpus_csv_cisco2)

In [ ]:
trainer1.train('/content/cisco_data_sent',
              learning_rate=0.05,
              mini_batch_size=50,
              max_epochs=5)
trainer2.train('/content/cisco_data_lstm',
              learning_rate=0.05,
              mini_batch_size=50,
              max_epochs=5)